In [1]:
!pip install timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import os
import timm

In [3]:
train = pd.read_csv("/kaggle/input/soil-classification/soil_classification-2025/train_labels.csv")
test = pd.read_csv("/kaggle/input/soil-classification/soil_classification-2025/test_ids.csv")

In [4]:
soil_mapping = {
    "Alluvial soil": 0,
    "Black Soil": 1,
    "Clay soil": 2,
    "Red soil": 3
}

train["soil_type"] = train["soil_type"].replace(soil_mapping)

/tmp/ipykernel_35/2526885511.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train["soil_type"] = train["soil_type"].replace(soil_mapping)


In [7]:
train

,image_id,soil_type
0,img_ed005410.jpg,0
1,img_0c5ecd2a.jpg,0
2,img_ed713bb5.jpg,0
3,img_12c58874.jpg,0
4,img_eff357af.jpg,0
...,...,...
1217,img_9ae546bd.jpg,1
1218,img_6e0b1b7c.jpg,1
1219,img_5c4372f8.jpg,1
1220,img_6f5bf9d2.jpg,1


In [8]:
train.describe()

,soil_type
count,1222.000000
mean,1.162848
std,1.197388
min,0.000000
25%,0.000000
50%,1.000000
75%,2.000000
max,3.000000


In [5]:
class ImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        path = self.data.loc[idx, 'image_id']
        label = int(self.data.loc[idx, 'soil_type'])
        img_path = f"/kaggle/input/soil-classification/soil_classification-2025/train/{path}"

        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

In [6]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(train, test_size=0.2, random_state=42, shuffle=True)

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


In [9]:
dataset = ImageDataset(dataframe = train_df, transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# model = models.resnet50(pretrained=True)
# model.fc = nn.Linear(model.fc.in_features, 4)

model = models.densenet121(pretrained=True)
model.classifier = nn.Linear(model.classifier.in_features, 4)

model = timm.create_model('vit_base_patch16_224', pretrained=True)
model.head = nn.Linear(model.head.in_features, 4)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader):.4f}")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 214MB/s]


Epoch 1/10, Loss: 1.0804
Epoch 2/10, Loss: 0.5070
Epoch 3/10, Loss: 0.2988
Epoch 4/10, Loss: 0.2157
Epoch 5/10, Loss: 0.1434
Epoch 6/10, Loss: 0.1178
Epoch 7/10, Loss: 0.1011
Epoch 8/10, Loss: 0.0669
Epoch 9/10, Loss: 0.0625
Epoch 10/10, Loss: 0.0499


In [10]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

test_dataset = ImageDataset(test_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()
model.to(device)

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

f1 = f1_score(all_labels, all_preds, average='macro')
print(f"Balanced F1 Score (macro): {f1:.4f}")


Test Accuracy: 98.78%
Balanced F1 Score (macro): 0.9878


In [23]:
class InferenceDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        path = self.data.loc[idx, 'image_id']
        img_path = f"/kaggle/input/soil-classification/soil_classification-2025/test/{path}"
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, img_path , path


In [24]:
inference_dataset = InferenceDataset(test, transform=transform)
inference_loader = DataLoader(inference_dataset, batch_size=32, shuffle=False)

model.eval()
model.to(device)

predicted_labels = []
image_ids = []

with torch.no_grad():
    for images, paths , ids in inference_loader:
        images = images.to(device)

        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)

        predicted_labels.extend(preds.cpu().numpy())
        image_ids.extend(ids)


In [25]:
results_df = pd.DataFrame({
    'image_id': image_ids,
    'predicted_label': predicted_labels
})

print(results_df.head())

            image_id  predicted_label
0  img_cdf80d6f.jpeg                0
1   img_c0142a80.jpg                0
2   img_91168fb0.jpg                0
3   img_9822190f.jpg                0
4  img_e5fc436c.jpeg                0


In [26]:
inv_soil_mapping = {v: k for k, v in soil_mapping.items()}
results_df['soil_type'] = results_df['predicted_label'].map(inv_soil_mapping)
results_df = results_df.drop(columns=['predicted_label'])

In [27]:
results_df

,image_id,soil_type
0,img_cdf80d6f.jpeg,Alluvial soil
1,img_c0142a80.jpg,Alluvial soil
2,img_91168fb0.jpg,Alluvial soil
3,img_9822190f.jpg,Alluvial soil
4,img_e5fc436c.jpeg,Alluvial soil
...,...,...
336,img_bc768d49.jpg,Black Soil
337,img_ddef2a37.jpg,Black Soil
338,img_be2e7e88.jpg,Black Soil
339,img_04f21bb9.jpg,Black Soil


In [28]:
results_df.to_csv('predictions_dense.csv', index=False)

In [29]:
pred_resnet = pd.read_csv('predictions_dense.csv')
pred_densenet = pd.read_csv('predictions_vit.csv')

assert all(pred_resnet['image_id'] == pred_densenet['image_id']), "Image paths do not align!"

comparison = pred_resnet['soil_type'] == pred_densenet['soil_type']

num_same = comparison.sum()
num_total = len(comparison)
num_diff = num_total - num_same

print(f"Total Predictions: {num_total}")
print(f"Same Predictions: {num_same}")
print(f"Different Predictions: {num_diff}")


Total Predictions: 341
Same Predictions: 332
Different Predictions: 9
